# Week 3: Filtering & Frequency Domain — Lab Practice

**Topics:** Image Resizing, Convolution, Smoothing, Frequency Domain & Sharpening

This notebook accompanies the Week 3 lecture slides. We will resize images with different interpolation methods, implement 2D convolution from scratch, apply smoothing and sharpening filters, and visualize the frequency domain — all hands-on.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import correlate

%matplotlib inline

### Environment setup

This notebook works both **locally** and on **Google Colab**.
- **Local**: images are loaded from the repository’s `images/` folder.
- **Colab**: images are automatically downloaded from GitHub on first run.

In [ ]:
import os, urllib.request

# Detect Google Colab
IN_COLAB = "google.colab" in str(get_ipython()) if hasattr(__builtins__, "__IPYTHON__") else False

# Image paths — we use a 256×256 version for faster processing
REPO_URL = "https://raw.githubusercontent.com/HyeongminLEE/image-processing-tutorial/main"
IMAGE_DIR = "images"
COLOR_NAME = "parrots_256.jpg"
GRAY_NAME = "parrots_256_gray.jpg"

if IN_COLAB:
    os.makedirs(IMAGE_DIR, exist_ok=True)
    for name in [COLOR_NAME, GRAY_NAME]:
        url = f"{REPO_URL}/{IMAGE_DIR}/{name}"
        dest = os.path.join(IMAGE_DIR, name)
        if not os.path.exists(dest):
            print(f"Downloading {name} ...")
            urllib.request.urlretrieve(url, dest)
    IMAGE_PATH = os.path.join(IMAGE_DIR, COLOR_NAME)
    GRAY_PATH = os.path.join(IMAGE_DIR, GRAY_NAME)
else:
    IMAGE_PATH = os.path.join("..", IMAGE_DIR, COLOR_NAME)
    GRAY_PATH = os.path.join("..", IMAGE_DIR, GRAY_NAME)

print(f"Image path: {IMAGE_PATH}")
print(f"Running on: {'Google Colab' if IN_COLAB else 'Local'}")

### Display helpers

We define utility functions used throughout this notebook. This week adds `show_kernel` for visualizing filter weights and `show_spectrum` for displaying magnitude spectra, alongside the familiar `show_images`.

In [ ]:
def show_images(*imgs, titles=None, scale=4):
    """Display images in a single row.

    Grayscale (2-D) arrays automatically use a gray colormap.
    *titles* is an optional list of strings, one per image.
    """
    n = len(imgs)
    fig, axes = plt.subplots(1, n, figsize=(scale * n, scale))
    if n == 1:
        axes = [axes]
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        if img.ndim == 2:
            ax.imshow(img, cmap="gray", vmin=0, vmax=255)
        else:
            ax.imshow(img)
        if titles:
            ax.set_title(titles[i])
        ax.axis("off")
    plt.tight_layout()
    plt.show()


def show_proportional(*imgs, titles=None, scale=6):
    """Display images vertically, each at its proportional size.

    *scale* sets the width (in inches) of the widest image.
    """
    n = len(imgs)
    max_w = max(img.shape[1] for img in imgs)
    px_per_inch = max_w / scale
    heights = [img.shape[0] / px_per_inch for img in imgs]
    gap = 0.35
    total_h = sum(heights) + gap * (n - 1)

    fig, axes = plt.subplots(n, 1, figsize=(scale, total_h),
                             gridspec_kw={"height_ratios": [img.shape[0] for img in imgs]})
    if n == 1:
        axes = [axes]
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        ax.set_anchor("W")
        if img.ndim == 2:
            ax.imshow(img, cmap="gray", vmin=0, vmax=255)
        else:
            ax.imshow(img)
        if titles:
            ax.set_title(titles[i], loc="left")
        ax.axis("off")
    fig.subplots_adjust(hspace=gap * n / sum(heights))
    plt.show()


def show_kernel(kernel, title="Kernel", ax=None):
    """Display a small kernel as an annotated heatmap."""
    standalone = ax is None
    if standalone:
        fig, ax = plt.subplots(figsize=(3, 3))
    vmax = np.max(np.abs(kernel))
    ax.imshow(kernel, cmap="coolwarm", vmin=-vmax, vmax=vmax)
    for i in range(kernel.shape[0]):
        for j in range(kernel.shape[1]):
            val = kernel[i, j]
            text = f"{val:.2f}" if val != int(val) else str(int(val))
            ax.text(j, i, text, ha="center", va="center", fontsize=10)
    ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])
    if standalone:
        plt.tight_layout()
        plt.show()


def show_kernels(*kernels, titles=None):
    """Display multiple kernels as annotated heatmaps in a row."""
    n = len(kernels)
    fig, axes = plt.subplots(1, n, figsize=(3 * n, 3))
    if n == 1:
        axes = [axes]
    for i, (ax, k) in enumerate(zip(axes, kernels)):
        show_kernel(k, title=titles[i] if titles else "Kernel", ax=ax)
    plt.tight_layout()
    plt.show()


def show_kernel_3d(kernel, title=""):
    """Display a kernel as a 3D surface plot."""
    k = kernel.shape[0] // 2
    ax_range = np.arange(-k, k + 1)
    xx, yy = np.meshgrid(ax_range, ax_range)
    fig = plt.figure(figsize=(6, 5))
    ax = fig.add_subplot(111, projection="3d")
    ax.plot_surface(xx, yy, kernel, cmap="viridis")
    if title:
        ax.set_title(title)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    plt.tight_layout()
    plt.show()


def show_maps(*maps, titles=None, scale=4):
    """Display 2D float arrays (masks, spectra, etc.) with auto grayscale mapping."""
    n = len(maps)
    fig, axes = plt.subplots(1, n, figsize=(scale * n, scale))
    if n == 1:
        axes = [axes]
    for i, (ax, m) in enumerate(zip(axes, maps)):
        ax.imshow(m, cmap="gray")
        if titles:
            ax.set_title(titles[i])
        ax.axis("off")
    plt.tight_layout()
    plt.show()


def show_spectrum(img, spectrum, title=""):
    """Display a grayscale image and its magnitude spectrum side by side.

    *spectrum* is a pre-computed 2D array (e.g., from log_magnitude_spectrum).
    """
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    axes[0].imshow(img, cmap="gray", vmin=0, vmax=255)
    axes[0].set_title(f"{title} (Spatial)" if title else "Spatial domain")
    axes[0].axis("off")
    axes[1].imshow(spectrum, cmap="gray")
    axes[1].set_title(f"{title} (Spectrum)" if title else "Magnitude spectrum")
    axes[1].axis("off")
    plt.tight_layout()
    plt.show()


def show_freq_response(responses, labels, xlim=None):
    """Plot 1D frequency response cross-sections through center."""
    fig, ax = plt.subplots(figsize=(6, 4))
    for resp, label in zip(responses, labels):
        center = resp.shape[0] // 2
        freq_axis = np.arange(resp.shape[1]) - resp.shape[1] // 2
        ax.plot(freq_axis, resp[center, :], label=label)
    ax.set_title("Frequency response (1D cross-section)")
    ax.set_xlabel("Frequency")
    ax.set_ylabel("Magnitude")
    ax.legend()
    if xlim:
        ax.set_xlim(xlim)
    plt.tight_layout()
    plt.show()


def rescale_for_display(img):
    """Rescale a float image to [0, 255] uint8 for display."""
    lo, hi = img.min(), img.max()
    return ((img - lo) / (hi - lo) * 255).astype(np.uint8)

---
## 1. Image Resizing & Interpolation

When we change an image’s dimensions, pixels must be **created** (upsampling) or **discarded** (downsampling). The interpolation method determines how new pixel values are estimated from existing ones.

In [ ]:
img_color = np.array(Image.open(IMAGE_PATH))
img_gray = np.array(Image.open(GRAY_PATH))

print(f"Color \u2014 shape: {img_color.shape}, dtype: {img_color.dtype}")
print(f"Gray  \u2014 shape: {img_gray.shape}, dtype: {img_gray.dtype}")

### Downsampling by pixel skipping

The simplest downsampling: take every $N$th pixel and discard the rest. This is just NumPy slicing: `img[::N, ::N]`.

Each image below is displayed at its **actual relative size** so you can see how much information is lost.

In [ ]:
factors = [1, 2, 4]
downsampled = [img_gray[::f, ::f] for f in factors]
titles = [f"{d.shape[0]}×{d.shape[1]}" if f == 1
          else f"{d.shape[0]}×{d.shape[1]} (1/{f})"
          for f, d in zip(factors, downsampled)]

show_proportional(*downsampled, titles=titles)

### Upsampling with different interpolation methods

To upsample, we must **estimate** new pixel values. Three common methods:
- **Nearest neighbor**: copy the closest pixel — fast but blocky
- **Bilinear**: weighted average of 4 neighbors — smooth but slightly blurry
- **Bicubic**: cubic polynomial through 16 neighbors — sharpest, most common default

Let’s downsample to a small size, then upsample back to see the differences.

In [ ]:
pil_gray = Image.open(GRAY_PATH)
H, W = img_gray.shape

# Downsample to 32×32
small = pil_gray.resize((32, 32), Image.NEAREST)

# Upsample back with different methods
methods = {"Nearest": Image.NEAREST, "Bilinear": Image.BILINEAR, "Bicubic": Image.BICUBIC}
upsampled = {name: np.array(small.resize((W, H), method)) for name, method in methods.items()}

show_images(img_gray, upsampled["Nearest"], upsampled["Bilinear"], upsampled["Bicubic"],
            titles=["Original", "Nearest", "Bilinear", "Bicubic"])

Let’s zoom in on a small region to see the interpolation artifacts more clearly.

In [ ]:
r1, r2, c1, c2 = 20, 90, 20, 90
crops = [img_gray[r1:r2, c1:c2]] + [upsampled[m][r1:r2, c1:c2] for m in methods]

show_images(*crops, titles=["Original"] + list(methods.keys()), scale=3)

---
## 2. Spatial Filtering & Convolution

In Week 2, we transformed each pixel **independently** (point processing: $g(x,y) = T[f(x,y)]$). Now we use **neighborhoods**: the output at each pixel depends on the pixel *and its surrounding region*.

The core operation is **convolution**: slide a small weight matrix (kernel) over the image, computing a weighted sum at each position.

$$g(x,y) = \sum_{s=-k}^{k}\sum_{t=-k}^{k} h(s,t) \cdot f(x+s,\, y+t)$$

> **Note:** Strictly speaking, this formula is *correlation*. True mathematical convolution flips the kernel first. In practice, all standard libraries (`scipy`, `PyTorch`, etc.) compute correlation and call it "convolution" — we follow that convention here.

### 2D convolution: step by step

Let's work through a small example: a 5×5 "image" convolved with a 3×3 kernel.

In [ ]:
small_img = np.array([
    [1,  2,  3,  4,  5],
    [6,  7,  8,  9,  10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, 20],
    [21, 22, 23, 24, 25]
], dtype=np.float64)

kernel_2d = np.array([
    [0, -1,  0],
    [-1, 5, -1],
    [0, -1,  0]
], dtype=np.float64)

print(f"Image (5\u00d75):\n{small_img.astype(int)}")
print()
show_kernel(kernel_2d, title="Kernel (3\u00d73)")

Let's compute one output pixel explicitly: extract the 3×3 neighborhood, multiply element-wise with the kernel, and sum.

In [ ]:
# Compute output at position (1, 1) — kernel centered at row=1, col=1
row, col = 1, 1
neighborhood = small_img[row:row + 3, col:col + 3]
products = neighborhood * kernel_2d

print(f"Neighborhood at ({row},{col}):\n{neighborhood.astype(int)}")
print(f"\nKernel:\n{kernel_2d.astype(int)}")
print(f"\nElement-wise products:\n{products.astype(int)}")
print(f"\nSum = {products.sum():.0f}")

### Implementing 2D convolution from scratch

Let’s write a full convolution function. This builds deep intuition for what every library function does under the hood.

In [ ]:
def conv2d(image, kernel, mode="same"):
    """2D convolution of a grayscale image with a kernel.

    Parameters
    ----------
    image  : 2D array (H, W)
    kernel : 2D array (kH, kW), must be odd-sized
    mode   : "valid" or "same"
    """
    img = image.astype(np.float64)
    k = kernel.astype(np.float64)
    kH, kW = k.shape

    if mode == "same":
        pH, pW = kH // 2, kW // 2
        img = np.pad(img, ((pH, pH), (pW, pW)), mode="constant", constant_values=0)
    elif mode != "valid":
        raise ValueError(f"Unknown mode: {mode}")

    H, W = img.shape
    outH, outW = H - kH + 1, W - kW + 1
    output = np.zeros((outH, outW), dtype=np.float64)

    for i in range(outH):
        for j in range(outW):
            output[i, j] = np.sum(img[i:i + kH, j:j + kW] * k)

    return output

In [ ]:
result_valid = conv2d(small_img, kernel_2d, mode="valid")
result_same = conv2d(small_img, kernel_2d, mode="same")

print(f"Input shape:        {small_img.shape}")
print(f"Kernel shape:       {kernel_2d.shape}")
print(f"Valid output shape: {result_valid.shape}  (5 - 3 + 1 = 3)")
print(f"Same output shape:  {result_same.shape}   (padded to match input)")
print(f"\nValid output:\n{result_valid.astype(int)}")
print(f"\nSame output:\n{result_same.astype(int)}")

### Verification against scipy

Let's verify our implementation matches `scipy.ndimage.correlate`.

In [ ]:
ours = conv2d(small_img, kernel_2d, mode="same")
scipy_result = correlate(small_img, kernel_2d, mode="constant", cval=0)

print(f"Our conv2d:\n{ours.astype(int)}")
print(f"\nscipy correlate:\n{scipy_result.astype(int)}")
print(f"\nMatch: {np.allclose(ours, scipy_result)}")

### Applying convolution to a real image

Let's apply the sharpening kernel to the grayscale parrot image.

> Since our from-scratch implementation uses Python loops, it is **slow** on full-size images. For the rest of this notebook we use `scipy.ndimage.correlate` for efficiency.

In [ ]:
sharpened = correlate(img_gray.astype(np.float64), kernel_2d, mode="constant", cval=0)
sharpened = np.clip(sharpened, 0, 255).astype(np.uint8)

show_images(img_gray, sharpened, titles=["Original", "Sharpened (kernel from above)"])

### Output size

The output dimensions depend on the padding mode:

| Mode | Padding $p$ | Output size |
|:---:|:---:|:---:|
| **Valid** | $0$ | $H - F + 1$ |
| **Same** | $\lfloor F/2 \rfloor$ | $H$ |
| **Full** | $F - 1$ | $H + F - 1$ |

General formula: $O = H + 2p - F + 1$, where $F$ is the kernel size.

In [ ]:
H, W = img_gray.shape
F = 5
test_kernel = np.ones((F, F)) / (F * F)

valid_out = conv2d(img_gray, test_kernel, mode="valid")
same_out  = conv2d(img_gray, test_kernel, mode="same")

print(f"Input:  {img_gray.shape}")
print(f"Kernel: {test_kernel.shape}")
print(f"Valid:  {valid_out.shape}  (expected {H - F + 1}×{W - F + 1})")
print(f"Same:   {same_out.shape}  (expected {H}×{W})")

### Exercises

**Exercise 2.1:** Use `scipy.ndimage.correlate` to apply the **identity kernel** $\begin{bmatrix}0&0&0\\0&1&0\\0&0&0\end{bmatrix}$ to the grayscale image (`img_gray`). Display the original and result side by side.

Then try a **shift kernel** — place the `1` at a different position in the 3×3 matrix (e.g., bottom-right corner) and observe the effect. What happens to the image, and why?

In [ ]:
# YOUR CODE HERE


# What happens to the image when you apply the shift kernel? Why?
# (You may write your answer in Korean.)
# 

---
## 3. Smoothing Filters

Smoothing filters blur the image by averaging each pixel with its neighbors. They are **low-pass** filters: they preserve gradual intensity changes (low frequencies) and suppress rapid changes like edges and noise (high frequencies).

Two main types:
- **Box filter**: all weights equal (simple average)
- **Gaussian filter**: closer neighbors get higher weight (bell curve)

### Box filter

The simplest smoothing kernel: every weight is $\frac{1}{N^2}$. A 3×3 box filter:

$$h = \frac{1}{9}\begin{bmatrix}1 & 1 & 1\\1 & 1 & 1\\1 & 1 & 1\end{bmatrix}$$

Each output pixel becomes the **average** of its $3 \times 3$ neighborhood.

In [ ]:
def make_box_kernel(size):
    """Create a box (mean) kernel of the given size."""
    return np.ones((size, size), dtype=np.float64) / (size * size)

box3 = make_box_kernel(3)
show_kernel(box3, title="3\u00d73 Box kernel")

In [ ]:
sizes = [3, 7, 15, 31]
blurred = []
for s in sizes:
    k = make_box_kernel(s)
    b = correlate(img_gray.astype(np.float64), k, mode="constant", cval=0)
    blurred.append(np.clip(b, 0, 255).astype(np.uint8))

show_images(img_gray, *blurred,
            titles=["Original"] + [f"Box {s}\u00d7{s}" for s in sizes])

### Gaussian filter

Instead of equal weights, a Gaussian kernel gives **higher weight to closer neighbors**:

$$h(s,t) = \frac{1}{2\pi\sigma^2}\exp\!\left(-\frac{s^2+t^2}{2\sigma^2}\right)$$

The parameter $\sigma$ controls the spread: larger $\sigma$ = wider kernel = more smoothing.

In [ ]:
def make_gaussian_kernel(size, sigma):
    """Create a 2D Gaussian kernel of the given size and sigma."""
    k = size // 2
    ax = np.arange(-k, k + 1, dtype=np.float64)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
    kernel /= kernel.sum()
    return kernel

g5 = make_gaussian_kernel(5, sigma=1.0)
print(f"Kernel shape: {g5.shape}")
print(f"Sum of weights: {g5.sum():.6f}")
show_kernel(g5, title="5\u00d75 Gaussian (\u03c3=1.0)")

In [ ]:
g15 = make_gaussian_kernel(15, sigma=3.0)
show_kernel_3d(g15, title="Gaussian kernel (\u03c3=3.0)")

In [ ]:
sigmas = [1, 3, 5, 10]
gaussian_blurred = []
for sigma in sigmas:
    size = int(6 * sigma + 1) | 1   # ensure odd size, ~6\u03c3 wide
    k = make_gaussian_kernel(size, sigma)
    b = correlate(img_gray.astype(np.float64), k, mode="constant", cval=0)
    gaussian_blurred.append(np.clip(b, 0, 255).astype(np.uint8))

show_images(img_gray, *gaussian_blurred,
            titles=["Original"] + [f"Gaussian \u03c3={s}" for s in sigmas])

### Box vs Gaussian comparison

At similar kernel sizes, Gaussian produces a **smoother, more natural** blur because nearby pixels contribute more than distant ones.

In [ ]:
box_result = correlate(img_gray.astype(np.float64), make_box_kernel(15), mode="constant", cval=0)
gauss_result = correlate(img_gray.astype(np.float64), make_gaussian_kernel(15, 3.0), mode="constant", cval=0)

box_result = np.clip(box_result, 0, 255).astype(np.uint8)
gauss_result = np.clip(gauss_result, 0, 255).astype(np.uint8)

show_images(img_gray, box_result, gauss_result,
            titles=["Original", "Box 15\u00d715", "Gaussian 15\u00d715 (\u03c3=3)"])

### Exercises

**Exercise 3.1:** Apply a 7×7 box filter to the color image (`img_color`). Since `correlate` works on 2D arrays, you need to process each R, G, B channel separately and recombine.

*Hint:* Use `make_box_kernel(7)` and `np.stack` to merge the channels back.

In [ ]:
# YOUR CODE HERE

**Exercise 3.2:** Smooth the grayscale image (`img_gray`) with a Gaussian filter ($\sigma=3$), then compute the **difference** between the original and the smoothed image. This difference is the **high-frequency detail** that smoothing removes.

Display three images: original, smoothed, and the difference.

*Hint:* The difference image will have negative values — you need to rescale it to [0, 255] for display. Think about how min-max stretching from Week 2 can help here.

In [ ]:
# YOUR CODE HERE

---
## 4. Frequency Domain & 2D FFT

The **Convolution Theorem** says: convolution in the spatial domain equals **multiplication** in the frequency domain.

$$f * h \;\longleftrightarrow\; F \cdot H$$

The **2D Discrete Fourier Transform (DFT)** decomposes an image into its constituent spatial frequencies. Low frequencies represent smooth, gradual changes; high frequencies represent edges and fine detail.

Key NumPy functions:
- `np.fft.fft2(img)` — compute the 2D DFT
- `np.fft.fftshift(F)` — move the DC component (average brightness) to the center
- `np.fft.ifft2(F)` — inverse DFT (back to spatial domain)

### Magnitude spectrum

To visualize the frequency content, we plot the **log-magnitude** of the DFT: $\log(1 + |F(u,v)|)$.

The log scaling compresses the huge dynamic range so that both the bright DC component and the faint high-frequency components are visible.

In [ ]:
def log_magnitude_spectrum(img):
    """Compute the centered log-magnitude spectrum of a grayscale image.

    Steps:
    1. 2D FFT            \u2192 np.fft.fft2
    2. Shift DC to center \u2192 np.fft.fftshift
    3. Log-compress       \u2192 log(1 + |F|)
    """
    F = np.fft.fft2(img.astype(np.float64))
    F_shifted = np.fft.fftshift(F)
    return np.log1p(np.abs(F_shifted))

In [ ]:
show_spectrum(img_gray, log_magnitude_spectrum(img_gray), title="Grayscale parrot")

### How image content maps to frequency content

Smooth regions concentrate energy at the **center** (low frequencies), while edges and textures spread energy **outward** (high frequencies). Let's compare the original with a heavily blurred version.

In [ ]:
smooth = correlate(img_gray.astype(np.float64),
                   make_gaussian_kernel(31, 5.0), mode="constant", cval=0)
smooth = np.clip(smooth, 0, 255).astype(np.uint8)

show_spectrum(img_gray, log_magnitude_spectrum(img_gray), title="Original")
show_spectrum(smooth, log_magnitude_spectrum(smooth), title="Heavily smoothed")

### Low-pass filtering in frequency domain

We can blur an image directly in the frequency domain:
1. Compute 2D DFT
2. Multiply by a **low-pass mask** (keeps center, blocks edges)
3. Inverse DFT back to spatial domain

The result is equivalent to spatial convolution with a smoothing filter.

In [ ]:
# Compute 2D DFT
F = np.fft.fft2(img_gray.astype(np.float64))
F_shifted = np.fft.fftshift(F)

# Create a circular low-pass mask
H, W = img_gray.shape
cy, cx = H // 2, W // 2
Y, X = np.ogrid[:H, :W]
radius = 30
mask_lp = ((X - cx)**2 + (Y - cy)**2 <= radius**2).astype(np.float64)

# Apply mask and inverse FFT
F_filtered = F_shifted * mask_lp
result_lp = np.fft.ifft2(np.fft.ifftshift(F_filtered)).real
result_lp = np.clip(result_lp, 0, 255).astype(np.uint8)

show_maps(mask_lp, titles=[f"Low-pass mask (r={radius})"])
show_images(img_gray, result_lp, titles=["Original", "Low-pass filtered"])
show_spectrum(img_gray, log_magnitude_spectrum(img_gray), title="Original")
show_spectrum(result_lp, log_magnitude_spectrum(result_lp), title="Low-pass filtered")

### High-pass filtering in frequency domain

The complement of low-pass is high-pass: $H_{\text{HP}} = 1 - H_{\text{LP}}$

High-pass filtering keeps edges and fine detail while removing smooth regions.

In [ ]:
mask_hp = 1 - mask_lp

F_hp = F_shifted * mask_hp
result_hp = np.fft.ifft2(np.fft.ifftshift(F_hp)).real
result_hp_display = rescale_for_display(result_hp)

show_maps(mask_hp, titles=["High-pass mask"])
show_images(img_gray, result_hp_display, titles=["Original", "High-pass filtered (edges)"])
show_spectrum(img_gray, log_magnitude_spectrum(img_gray), title="Original")
show_spectrum(result_hp_display, log_magnitude_spectrum(result_hp_display), title="High-pass filtered")

### Exercises

**Exercise 4.1:** Experiment with different **low-pass cutoff radii** (5, 15, 30, 60) on the grayscale image (`img_gray`). For each radius, create a circular mask, apply it in the frequency domain, and display the result. Show all four filtered images in a single row.

*Hint:* Reuse the FFT and mask construction pattern from the demo above, changing only the `radius`.

What happens as the radius increases? How does this relate to the kernel size of spatial smoothing?

In [ ]:
# YOUR CODE HERE


# What happens as the radius increases? How does it relate to the kernel size of spatial smoothing?
# (You may write your answer in Korean.)
# 

**Exercise 4.2:** Explore what happens when you **boost a single frequency component** in the spectrum.

1. Compute the FFT of `img_gray` (use `np.fft.fft2` and `np.fft.fftshift`)
2. Add a large value (e.g., `500000`) at position `(50, 50)` and its symmetric counterpart `(H-50, W-50)` — this keeps the result real-valued
3. Reconstruct the image with `np.fft.ifftshift` → `np.fft.ifft2`, take the real part, and clip to [0, 255]
4. Display the original and modified images side by side

Try a few different locations (vertical offset, horizontal offset, diagonal) and distances from center. How does the position of the boosted point relate to the wave pattern you see?

*Hint:* The symmetric counterpart ensures the result stays real-valued. For a point at `(cy - dy, cx - dx)`, its counterpart is `(cy + dy, cx + dx)`.

In [ ]:
# YOUR CODE HERE


# How does the point's position relate to the direction and frequency of the wave?
# (You may write your answer in Korean.)
# 

---
## 5. Sharpening Filters

Sharpening enhances edges and fine detail. The key insight:

$$\text{High-pass} = \text{Original} - \text{Low-pass (blurred)}$$

Therefore:

$$\text{Sharpened} = \text{Original} + \alpha \cdot \text{High-pass}$$

Two common approaches:
1. **Laplacian**: a single kernel that directly computes the second derivative (edges)
2. **Unsharp masking**: blur the image, subtract the blur to get edges, add them back

### Laplacian filter

The Laplacian kernel detects edges by computing the **second derivative** of the image. Its weights sum to **zero**, so flat regions produce zero output — only intensity changes survive.

Two variants:
- **4-connected**: $\begin{bmatrix}0 & 1 & 0\\1 & -4 & 1\\0 & 1 & 0\end{bmatrix}$ — horizontal & vertical edges
- **8-connected**: $\begin{bmatrix}1 & 1 & 1\\1 & -8 & 1\\1 & 1 & 1\end{bmatrix}$ — includes diagonal edges

In [ ]:
lap4 = np.array([[0,  1, 0],
                 [1, -4, 1],
                 [0,  1, 0]], dtype=np.float64)

lap8 = np.array([[1,  1, 1],
                 [1, -8, 1],
                 [1,  1, 1]], dtype=np.float64)

show_kernels(lap4, lap8, titles=["Laplacian (4-connected)", "Laplacian (8-connected)"])

print(f"4-connected sum: {lap4.sum():.0f}")
print(f"8-connected sum: {lap8.sum():.0f}")

In [ ]:
edges4 = correlate(img_gray.astype(np.float64), lap4, mode="constant", cval=0)
edges8 = correlate(img_gray.astype(np.float64), lap8, mode="constant", cval=0)

show_images(img_gray, rescale_for_display(edges4), rescale_for_display(edges8),
            titles=["Original", "Laplacian 4-connected", "Laplacian 8-connected"])

### Sharpening with the Laplacian

To sharpen, **subtract** the Laplacian (edge map) from the original:

$$\text{Sharpened} = \text{Original} - \text{Laplacian}$$

(We subtract because the standard Laplacian has a **negative** center weight. Equivalently, some formulations negate the kernel and add instead.)

In [ ]:
sharpened_lap = img_gray.astype(np.float64) - edges4
sharpened_lap = np.clip(sharpened_lap, 0, 255).astype(np.uint8)

show_images(img_gray, rescale_for_display(edges4), sharpened_lap,
            titles=["Original", "Laplacian edges", "Sharpened (orig \u2212 Laplacian)"])

### Unsharp masking

A more flexible sharpening method with a tunable strength parameter $\alpha$:

$$\text{Sharpened} = \text{Original} + \alpha \cdot (\text{Original} - \text{Blurred})$$

The term $(\text{Original} - \text{Blurred})$ extracts the high-frequency detail. Adding it back at strength $\alpha$ enhances edges.

In [ ]:
def unsharp_mask(img, sigma=3.0, alpha=1.0):
    """Apply unsharp masking to a grayscale image."""
    size = int(6 * sigma + 1) | 1
    kernel = make_gaussian_kernel(size, sigma)
    blurred = correlate(img.astype(np.float64), kernel, mode="constant", cval=0)
    detail = img.astype(np.float64) - blurred
    sharpened = img.astype(np.float64) + alpha * detail
    return np.clip(sharpened, 0, 255).astype(np.uint8)

In [ ]:
alphas = [0.5, 1.0, 2.0, 4.0]
results = [unsharp_mask(img_gray, sigma=3.0, alpha=a) for a in alphas]

show_images(img_gray, *results,
            titles=["Original"] + [f"\u03b1 = {a}" for a in alphas])

Let’s also visualize the **detail** (high-pass) image that unsharp masking extracts.

In [ ]:
size = int(6 * 3.0 + 1) | 1
k = make_gaussian_kernel(size, 3.0)
blurred_for_detail = correlate(img_gray.astype(np.float64), k, mode="constant", cval=0)
detail = img_gray.astype(np.float64) - blurred_for_detail

show_images(img_gray,
            np.clip(blurred_for_detail, 0, 255).astype(np.uint8),
            rescale_for_display(detail),
            titles=["Original", "Blurred (low-pass)", "Detail (high-pass)"])

### Exercises

**Exercise 5.1:** Build a simple **image enhancement pipeline** on the grayscale image (`img_gray`) that combines Weeks 2 and 3:
1. Apply **gamma correction** ($\gamma = 0.7$) to brighten dark regions
2. Apply **unsharp masking** ($\sigma=2, \alpha=1.0$) to sharpen

Display three images side by side: Original, After gamma, After gamma + sharpening.

In [ ]:
# YOUR CODE HERE

**Exercise 5.2:** Investigate what happens when you sharpen a **noisy** image.

1. Add Gaussian noise to `img_gray`: generate random values with `np.random.normal(0, 20, img_gray.shape)`, add to the image, and clip to [0, 255]
2. Apply unsharp masking ($\sigma=2, \alpha=2.0$) to both the **clean** and **noisy** images
3. Display four images: clean original, sharpened clean, noisy original, sharpened noisy

What happens to the noise after sharpening? Why does this occur?

*Hint:* Think about what frequency range noise occupies, and what sharpening does in the frequency domain.

In [ ]:
# YOUR CODE HERE


# What happens to the noise after sharpening? Why does this occur?
# (You may write your answer in Korean.)
# 